# Read all Tables from the Web page

In [ ]:
# Read all Tables from the Web page
import pandas as pd
dfHTML = pd.read_html("http://www.stockq.org/market/asia.php")

# Find out which item is our target Table in dfHTML
for i, df in enumerate(dfHTML):
    print(i, df.shape)

0 (1, 1)
1 (2, 1)
2 (1, 2)
3 (1, 2)
4 (1, 2)
5 (1, 1)
6 (1, 1)
7 (1, 1)
8 (1, 2)
9 (34, 9)
10 (3, 1)


In [ ]:
# Print the head of dfHTML[9]
print(dfHTML[9].head())

# Read the Target Table by Pandas
asia_stocks = dfHTML[9].loc[2:, :5]

                        0                       1                       2  \
0  亞洲股市行情 (Asian Markets)  亞洲股市行情 (Asian Markets)  亞洲股市行情 (Asian Markets)   
1                      股市                      指數                      漲跌   
2                     紐西蘭                11916.78                  -56.81   
3                      澳洲                 8081.20                   36.30   
4                   日經225                39759.50                  445.00   

                        3                       4                       5  \
0  亞洲股市行情 (Asian Markets)  亞洲股市行情 (Asian Markets)  亞洲股市行情 (Asian Markets)   
1                      比例                      最高                      最低   
2                  -0.47%                11973.59                11892.62   
3                   0.45%                 8091.10                 8044.90   
4                   1.13%                39783.50                39463.50   

                        6                       7                       8 

In [ ]:
# Print the table
print(asia_stocks)

           0         1       2       3         4         5
2        紐西蘭  11916.78  -56.81  -0.47%  11973.59  11892.62
3         澳洲   8081.20   36.30   0.45%   8091.10   8044.90
4      日經225  39759.50  445.00   1.13%  39783.50  39463.50
5       東證股價   2754.68   26.36   0.97%   2755.08   2732.64
6      東證100   1896.20   19.84   1.06%   1896.84   1879.94
7      東證500   2156.54   21.02   0.98%   2156.66   2138.71
8     東證1000   2607.72   25.06   0.97%   2608.06   2586.62
9         韓國   2705.14  -12.51  -0.46%   2741.23   2703.96
10      台灣加權  20796.20  378.50   1.85%         -         -
11      台灣店頭    254.93   -1.00  -0.39%    257.07    254.58
12      上海綜合   3049.19    2.14   0.07%   3051.59   3033.21
13      上海A股   3196.70    2.16   0.07%   3199.27   3179.94
14      上海B股    257.67    2.87   1.12%    258.14    254.91
15      深圳A股   1823.61    7.25   0.40%    1828.4   1809.97
16      深圳B股   1083.27    -2.9  -0.27%   1089.56    1083.2
17     滬深300   3525.36  -11.05  -0.31%   3543.71   3518.

# Web Crewler


## 抓取 PTT 八卦版原始內容

In [ ]:
# 爬蟲設定
import requests
ptt_url = "https://www.ptt.cc/bbs/Gossiping/index.html"

# 開始抓取
ptt_text = requests.get(ptt_url, cookies={'over18': '1'}).text
print(ptt_text)

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 Gossiping 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print">




	</head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/co

## 將抓取到的內容，整理成 JSON 格式

In [ ]:
# 剖析抓到的原始資料
from bs4 import BeautifulSoup
import json

soup = BeautifulSoup(ptt_text, 'html.parser')
articles = []

# 每篇文章都放在 <div class="r-ent"> ~ </div> 之間
for div in soup.find_all("div", class_="r-ent"):
  try:
    # 標題放在 <div class="title"> ~ </div> 之間
    title = div.find("div", class_="title").text.strip()
    # 網址放在 <a> 裡面的 href 屬性內，且缺乏首頁網址
    href = "https://www.ptt.cc" + div.find("a")['href']
    # 作者放在 <div class="author"> ~ </div> 之間
    author = div.find("div", class_="author").text.strip()
    # 日期放在 <div class="date"> ~ </div> 之間
    date = div.find("div", class_="date").text.strip()
    # 推文數放在 <div class="nrec"> ~ </div> 之間，且需轉成整數
    push_count = div.find("div", class_="nrec").text.strip()
    push_count = int(push_count) if push_count.isdigit() else 0

    # 將抓到的內容，存入 articles 串列中
    articles.append({
      "title": title,
      "href": href,
      "author": author,
      "push_count": push_count,
      "date": date
    })
  except:
    continue

# 將 articles 串列，轉換成 JSON 格式印出
print(json.dumps(articles, indent=2, ensure_ascii=False))

[
  {
    "title": "[問卦] 欸欸 消防隊會不會被民意逼去冒死救貓啊?",
    "href": "https://www.ptt.cc/bbs/Gossiping/M.1712648494.A.608.html",
    "author": "zxc2331189",
    "push_count": 0,
    "date": "4/09"
  },
  {
    "title": "[問卦] 超商的即期品為什麼不乾脆5折",
    "href": "https://www.ptt.cc/bbs/Gossiping/M.1712648550.A.1B0.html",
    "author": "Fucker5566",
    "push_count": 0,
    "date": "4/09"
  },
  {
    "title": "[問卦] 683T張順瑋退休了沒？",
    "href": "https://www.ptt.cc/bbs/Gossiping/M.1712648576.A.98F.html",
    "author": "leo921080931",
    "push_count": 1,
    "date": "4/09"
  },
  {
    "title": "[問卦] 16歲就可以合意性交了不是嗎？",
    "href": "https://www.ptt.cc/bbs/Gossiping/M.1712648617.A.12C.html",
    "author": "eco0701",
    "push_count": 3,
    "date": "4/09"
  },
  {
    "title": "[問卦]葉倫 布林肯 蕭茲 普丁 法國外長 訪中??",
    "href": "https://www.ptt.cc/bbs/Gossiping/M.1712648709.A.614.html",
    "author": "a874622",
    "push_count": 1,
    "date": "4/09"
  },
  {
    "title": "[問卦] 所以嘉義地震沒事了嗎",
    "href": "https://www.ptt.

## 將 JSON 存入 MongoDB


### 環境設定

In [ ]:
# 下載免費開源的 MongoDB 社群版
# 可從 https://www.mongodb.com/download-center/community/releases 查詢你要下載的版本
# 需與 Google Colab 當前使用的作業系統一致（Ubuntu 22.04）
!wget -qO - https://fastdl.mongodb.org/linux/mongodb-linux-x86_64-ubuntu2204-7.0.7.tgz | tar -xzv
!mv mongodb-linux-x86_64-ubuntu2204-7.0.7/bin/* /usr/bin/

mongodb-linux-x86_64-ubuntu2204-7.0.7/LICENSE-Community.txt
mongodb-linux-x86_64-ubuntu2204-7.0.7/MPL-2
mongodb-linux-x86_64-ubuntu2204-7.0.7/README
mongodb-linux-x86_64-ubuntu2204-7.0.7/THIRD-PARTY-NOTICES
mongodb-linux-x86_64-ubuntu2204-7.0.7/bin/install_compass
mongodb-linux-x86_64-ubuntu2204-7.0.7/bin/mongod
mongodb-linux-x86_64-ubuntu2204-7.0.7/bin/mongos


In [ ]:
# 下載與 MongoDB 版本相應的金鑰，保證 MongoDB 連線的安全
# 可以上 https://pgp.mongodb.com/ 查詢欲下載金鑰的完整檔名
!wget https://www.mongodb.org/static/pgp/server-7.0.asc
!sudo apt-key add server-7.0.asc

--2024-03-30 08:55:20--  https://www.mongodb.org/static/pgp/server-7.0.asc
Resolving www.mongodb.org (www.mongodb.org)... 52.206.222.245, 54.175.147.155, 52.21.89.200
Connecting to www.mongodb.org (www.mongodb.org)|52.206.222.245|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pgp.mongodb.com/server-7.0.asc [following]
--2024-03-30 08:55:20--  https://pgp.mongodb.com/server-7.0.asc
Resolving pgp.mongodb.com (pgp.mongodb.com)... 18.160.41.70, 18.160.41.8, 18.160.41.90, ...
Connecting to pgp.mongodb.com (pgp.mongodb.com)|18.160.41.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1674 (1.6K) [binary/octet-stream]
Saving to: ‘server-7.0.asc’

server-7.0.asc      100%[===================>]   1.63K  --.-KB/s    in 0s      

2024-03-30 08:55:20 (390 MB/s) - ‘server-7.0.asc’ saved [1674/1674]

OK


In [ ]:
# 建造 MongoDB 資料存放處（路徑、名稱可自訂）
!mkdir -p /content/data/db

# 啟動 MongoDB 伺服器（Mongo Daemon）於本地端（localhost）
!mongod --fork --logpath /var/log/mongodb.log --dbpath /content/data/db --bind_ip localhost

about to fork child process, waiting until server is ready for connections.
forked process: 4131
child process started successfully, parent exiting


In [ ]:
# 確認 MongoDB 正在執行中
!mongod --sysinfo

{
    "sysinfo": {
        "_SC_PAGE_SIZE": 4096,
        "_SC_PHYS_PAGES": 3322618,
        "_SC_AVPHYS_PAGES": 2226431
    }
}


In [ ]:
# 安裝 PyMongo 函式庫
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 22.0 MB/s eta 0:00:00


### 存取 MongoDB 資料庫

In [ ]:
# 將站點資訊存入 MongoDB 資料庫中
from pymongo import MongoClient
client = MongoClient()
gossip = client.ptt.gossiping

if isinstance(articles, list):
    gossip.insert_many(articles)
else:
    gossip.insert_one(articles)

In [ ]:
# 從資料庫找出有任何推文的帖子
cursor = gossip.find({"push_count": {"$gt": 0}})

# 將這些帖子印出來驗證一下
for document in cursor:
    # Show "標題", "網址", and "推文數"
    print("標題：", document["title"])
    print("網址：", document["href"])
    print("推文數：", document["push_count"])
    print("---------------------------------")

# 計算總共有幾篇帖子推文數 > 0 的
count = gossip.count_documents({"push_count": {"$gt": 0}})
print("有推文帖子總數：", count)

標題： [問卦] 張學友演唱會沒有清票代表什麼？
網址： https://www.ptt.cc/bbs/Gossiping/M.1711787192.A.8BD.html
推文數： 4
---------------------------------
標題： Re: [問卦] 這女生很漂亮，為何還會被渣男多次劈
網址： https://www.ptt.cc/bbs/Gossiping/M.1711787214.A.285.html
推文數： 5
---------------------------------
標題： [問卦] 朋友媽媽去買蕃茄丁 該怎麼辦？
網址： https://www.ptt.cc/bbs/Gossiping/M.1711787262.A.777.html
推文數： 1
---------------------------------
標題： [問卦] 中國人一聽到台語有混日語就崩潰？
網址： https://www.ptt.cc/bbs/Gossiping/M.1711787294.A.531.html
推文數： 11
---------------------------------
標題： [新聞] 小行星獲命名賴和 賴清德：宇宙有光芒來
網址： https://www.ptt.cc/bbs/Gossiping/M.1711787314.A.229.html
推文數： 2
---------------------------------
標題： Re: [問卦] 國共內戰國府有派系內鬥，共產黨沒有？
網址： https://www.ptt.cc/bbs/Gossiping/M.1711787417.A.73F.html
推文數： 2
---------------------------------
標題： [問卦] 很多俄羅斯妹子想嫁台灣人？
網址： https://www.ptt.cc/bbs/Gossiping/M.1711787551.A.A64.html
推文數： 2
---------------------------------
標題： [問卦] 高利貸合法的原因是不是用別的名義
網址： https://www.ptt.cc/bbs/Gossiping/M.1711787573.A.01A.html
推文數： 2
